<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

(Schräge, 2003) Suppose you manage your company's strategic planning
department. There are eight analysts in the department. Your department is
about to move into a new suite of offices. There are four offices in the new suite and you need to match up your analysts into four pairs, so that each pair can be
assigned to one of the new offices. 

Based on past observations, you know some
of the analysts work better together than they do with others. In the interest of
departmental peace, you would like to come up with a pairing of analysts
that results in minimal potential conflicts. To this goal, you have come up with
a rating system for pairing your analysts. The scale runs from 1 to 10, with a 1
rating for a pair meaning the two get along fantastically, whereas all sharp
objects should be removed from the pair's office in anticipation of mayhem
for a rating of 10. The ratings appear in Table.


Since the pairing of analyst / with analyst J is indistinguishable from the
pairing of J with /, we have only included the above diagonal elements in the
table. Our problem is to find the pairings of analysts that minimizes the sum
of the incompatibility ratings of the paired analysts.

In [1]:
!pip install pulp
from pulp import *
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Matching Employees", LpMaximize)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [2]:
employees = range(8)
group = 4

\begin{equation} \label{y_var_def}
    y_{i,j} = \left \{ \begin{array}{ll}
      1 & \mbox{if employee $i$ is paired with employee $j$}\\
      0 & \mbox{otherwise.} \end{array} \right.
  \end{equation}

In [3]:
#Define the variables
y = LpVariable.dicts("pair", [(i,j) for i in employees for j in employees] ,cat='Binary')

In [4]:
import numpy as np 
import pandas as pd
np.random.seed(0)

c = np.random.randint(0,10, (8,8))
np.fill_diagonal(c,0)

c

array([[0, 0, 3, 3, 7, 9, 3, 5],
       [2, 0, 7, 6, 8, 8, 1, 6],
       [7, 7, 0, 1, 5, 9, 8, 9],
       [4, 3, 0, 0, 5, 0, 2, 3],
       [8, 1, 3, 3, 0, 7, 0, 1],
       [9, 9, 0, 4, 7, 0, 2, 7],
       [2, 0, 0, 4, 5, 5, 0, 8],
       [4, 1, 4, 9, 8, 1, 1, 0]])

In [5]:
names = ['Ben','Kate','Thinh','Jorge','Alfredo','Francisco','Olivia','Chris']
match_info = pd.DataFrame(c, index=names, columns=names)

match_info

,Ben,Kate,Thinh,Jorge,Alfredo,Francisco,Olivia,Chris
Ben,0,0,3,3,7,9,3,5
Kate,2,0,7,6,8,8,1,6
Thinh,7,7,0,1,5,9,8,9
Jorge,4,3,0,0,5,0,2,3
Alfredo,8,1,3,3,0,7,0,1
Francisco,9,9,0,4,7,0,2,7
Olivia,2,0,0,4,5,5,0,8
Chris,4,1,4,9,8,1,1,0


In [6]:
c[0,1]

0

Maximize $\sum_{j=0}^n\sum_{i=0}^n (c_{i,j}+c_{j,i}) \cdot y_{i,j}$

In [7]:
prob += lpSum([(c[i][j] + c[j][i]) * y[(i,j)] for i in employees for j in employees])

Employee $i$ is paired with no more than one employee $j$

\begin{equation}
\sum_{i} y_{i,j}\leq 1 \; \forall j \;\in employees
\end{equation}

Employee $j$ is paired with no more than one employee $i$

\begin{equation}
\sum_{i} y_{j,i} \leq 1 \; \forall j \;\in employees
\end{equation}

Pairing between employee $i$ and $j$ also means to pair between employee $j$ and $i$

\begin{equation}
y_{i,j} + y_{j,i} \leq 1 \; \forall i,j \;\in employees
\end{equation}

There is a total of 4 pairs

\begin{equation}
\sum_j\sum_{i} y_{i,j} = 4
\end{equation}

In [8]:
for i in employees:
    prob += lpSum(y[(i,j)] for j in employees) <= 1
    prob += lpSum(y[(j,i)] for j in employees) <= 1
    prob += lpSum(y[(i,j)] for j in employees)+ lpSum(y[(j,i)] for j in employees) <= 1


prob += lpSum(y[(i,j)] for i in employees for j in employees) == 4

In [9]:
prob.solve()

1

In [14]:
print("Finish matching!\n")
for i in employees:
    for j in employees:
        if y[(i,j)].varValue == 1:
            print('{} and {} with preference score {} and {}. Total score: {}'.format(names[i],names[j],c[i,j], c[j,i], c[i,j] +c[j,i]))

Finish matching!

Ben and Alfredo with preference score 7 and 8. Total score: 15
Kate and Francisco with preference score 8 and 9. Total score: 17
Jorge and Chris with preference score 3 and 9. Total score: 12
Olivia and Thinh with preference score 0 and 8. Total score: 8


In [11]:
prob.writeLP('employers.txt')

[pair_(0,_0),
 pair_(0,_1),
 pair_(0,_2),
 pair_(0,_3),
 pair_(0,_4),
 pair_(0,_5),
 pair_(0,_6),
 pair_(0,_7),
 pair_(1,_0),
 pair_(1,_1),
 pair_(1,_2),
 pair_(1,_3),
 pair_(1,_4),
 pair_(1,_5),
 pair_(1,_6),
 pair_(1,_7),
 pair_(2,_0),
 pair_(2,_1),
 pair_(2,_2),
 pair_(2,_3),
 pair_(2,_4),
 pair_(2,_5),
 pair_(2,_6),
 pair_(2,_7),
 pair_(3,_0),
 pair_(3,_1),
 pair_(3,_2),
 pair_(3,_3),
 pair_(3,_4),
 pair_(3,_5),
 pair_(3,_6),
 pair_(3,_7),
 pair_(4,_0),
 pair_(4,_1),
 pair_(4,_2),
 pair_(4,_3),
 pair_(4,_4),
 pair_(4,_5),
 pair_(4,_6),
 pair_(4,_7),
 pair_(5,_0),
 pair_(5,_1),
 pair_(5,_2),
 pair_(5,_3),
 pair_(5,_4),
 pair_(5,_5),
 pair_(5,_6),
 pair_(5,_7),
 pair_(6,_0),
 pair_(6,_1),
 pair_(6,_2),
 pair_(6,_3),
 pair_(6,_4),
 pair_(6,_5),
 pair_(6,_6),
 pair_(6,_7),
 pair_(7,_0),
 pair_(7,_1),
 pair_(7,_2),
 pair_(7,_3),
 pair_(7,_4),
 pair_(7,_5),
 pair_(7,_6),
 pair_(7,_7)]

In [12]:
pulp.value(prob.objective)

26.0

In [13]:
pulp.LpStatus[prob.status]

'Optimal'